In [13]:
# import packages
from covid19dh import covid19
from datetime import date
from Get_covid_data import get_data
import pymc3 as pm
import pandas as pd 
import numpy as np 
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import os 
import theano
import random
from sklearn.preprocessing import MinMaxScaler
# import functions from fns file 
import fns as f

Load covid data, create subset/new columns and plot

In [14]:
#Get data
data = get_data(level = 2, start = date(2020,1,1)) #can get more or less data here.
#group_by introduce lag (new infected from commulative)
data["new_infected"] = data.groupby(["administrative_area_level_2"])["confirmed"].diff()
data = data[data["new_infected"].notna()]
##Create subset from 5 states
#subset = data[data['administrative_area_level_2'].isin(["Florida"])]
##Check data
#sns.lineplot(data = data, x = "date", y = "new_infected", hue = "administrative_area_level_2")



Prep data and create train/test split

In [16]:
# specify x, y & grouping variable in orig. data &
# the name that will be used going forward (can be the same). 
y = "new_infected"
x_old = "date"
idx_old = "administrative_area_level_2"
x_new = "date_2"
idx_new = "idx"

# get idx for each group (e.g. country) and zero-index for time variable (e.g. year).
d = f.get_idx(data, idx_old, x_old, idx_new, x_new)

# use test/train split function
train, test = f.train_test(d, x_new) # check doc-string.

#Scale values:
scaler = MinMaxScaler()
train['new_infected'] = scaler.fit_transform(train['new_infected'].values.reshape(-1,1))
test['new_infected'] = scaler.fit_transform(test['new_infected'].values.reshape(-1,1))

# we need N here as well for shape.
N = len(np.unique(train[idx_new]))

<ipython-input-16-068e3227cac0>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['new_infected'] = scaler.fit_transform(train['new_infected'].values.reshape(-1,1))
<ipython-input-16-068e3227cac0>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['new_infected'] = scaler.fit_transform(test['new_infected'].values.reshape(-1,1))


Build model

In [19]:
# pooled model
with pm.Model() as m1: 
    
    # hyper-priors
    α_μ = pm.Normal('α_μ', mu=0, sd=1)
    α_σ = pm.HalfNormal('α_σ', 1)
    β_μ = pm.Normal('β_μ', mu=0, sd=1)
    β_σ = pm.HalfNormal('β_σ', sd=1)
    
    # priors
    α = pm.Normal('α', mu=α_μ, sd=α_σ, shape=N)
    β = pm.Normal('β', mu=β_μ, sd=β_σ, shape=N)
    ε = pm.HalfCauchy('ε', 1)
    
    # containers 
    x_ = pm.Data('m1_x', train[x_new].values) #Year_ as data container (can be changed). 
    idx_ = pm.Data('m1_idx', train[idx_new].values)
    
    y_pred = pm.Normal('y_pred',
                       mu=α[idx_] + β[idx_] * x_,
                       sd=ε, observed=train[y].values)
    
    # trace 
    m1_trace = pm.sample(2000, chains = 1, return_inferencedata = True,
                        target_accept = .99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [ε, β, α, β_σ, β_μ, α_σ, α_μ]


C:\Users\emilr\Anaconda3\envs\CM\lib\site-packages\pymc3\step_methods\hmc\quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
C:\Users\emilr\Anaconda3\envs\CM\lib\site-packages\pymc3\step_methods\hmc\quadpotential.py:203: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)


ValueError: Mass matrix contains zeros on the diagonal. 
The derivative of RV `α`.ravel()[0] is zero.
The derivative of RV `α`.ravel()[1] is zero.
The derivative of RV `α`.ravel()[2] is zero.
The derivative of RV `α`.ravel()[3] is zero.
The derivative of RV `α`.ravel()[4] is zero.
The derivative of RV `α`.ravel()[6] is zero.
The derivative of RV `α`.ravel()[7] is zero.
The derivative of RV `α`.ravel()[8] is zero.
The derivative of RV `α`.ravel()[10] is zero.
The derivative of RV `α`.ravel()[11] is zero.
The derivative of RV `α`.ravel()[12] is zero.
The derivative of RV `α`.ravel()[13] is zero.
The derivative of RV `α`.ravel()[14] is zero.
The derivative of RV `α`.ravel()[15] is zero.
The derivative of RV `α`.ravel()[16] is zero.
The derivative of RV `α`.ravel()[17] is zero.
The derivative of RV `α`.ravel()[18] is zero.
The derivative of RV `α`.ravel()[19] is zero.
The derivative of RV `α`.ravel()[20] is zero.
The derivative of RV `α`.ravel()[21] is zero.
The derivative of RV `α`.ravel(

In [ ]:
# check trace
az.plot_trace(m1_trace)

In [ ]:
# posterior predictive on new data. 
predictions = f.pp_test(
    m1, 
    m1_trace, 
    {'m1_x': test[x_new].values, 'm1_idx': test[idx_new].values},
    params = ["α", "β", "y_pred"]
    )

In [20]:
# plot posterior predictive on new data. 
f.plot_pp(predictions, train, test, d, x_new, y, idx_old)

NameError: name 'predictions' is not defined